# Timeseries Angle Encoding
*TS utilities in Qiskit*

By: Jacob Cybulski<br>
Date: August 2023 - March 2024<br>
Aims: The goal of this notebook is to test a Angle encodong utilities.

### Angle encoding
Deltas between consecutive time series values have been angle encoded. In the context of a quibit representation (see the Figure), the encoding assumes zero to be encoded as H state, negative values to be rotations up, while positive values as rotations down. This encoding allows cumulative sequence calculations and easy value decoding upong the qubit measurements. Should there be huge voilatility in data, additional scaling has been added to shrink the region of valid angular qubit positions. 

<div>
    <!--img src="attachment:qae_fig2_wide.png" width="1000"-->
    <img src="../images/seq-value-encoding.png" width="300">
</div>

**Figure: Sequence value coding as qubit angular rotations**

In [1]:
%%html
<style>
table {float:left}
</style>

In [2]:
import sys
sys.path.append('.')
sys.path.append('..')
sys.path

['/home/jacob/Dropbox/Dev/Python/quantum_projects/ts/ts_qae_sidekick/utils',
 '/home/jacob/miniconda3/envs/qiskit-gpu/lib/python310.zip',
 '/home/jacob/miniconda3/envs/qiskit-gpu/lib/python3.10',
 '/home/jacob/miniconda3/envs/qiskit-gpu/lib/python3.10/lib-dynload',
 '',
 '/home/jacob/miniconda3/envs/qiskit-gpu/lib/python3.10/site-packages',
 '.',
 '..']

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pylab
import math
from IPython.display import clear_output
%matplotlib inline

from utils.Angles import *

In [4]:
### Test
print('Encoding:')
print_ts_relang_encode_val(0)
print_ts_relang_encode_val(0.5)
print_ts_relang_encode_val(0.7)
print_ts_relang_encode_val(0.25)
print_ts_relang_encode_val(1.2) # Out of [0..pi] range
print_ts_relang_encode_val(-0.7) # Out of [0..pi] range

print('\nDecoding:')
print_ts_relang_decode_val(np.pi)
print_ts_relang_decode_val(0*np.pi)
print_ts_relang_decode_val(np.pi/4)
print_ts_relang_decode_val(-np.pi/2)
print_ts_relang_decode_val(np.pi/8)
print_ts_relang_decode_val(-0.9*np.pi) # Out of [0..pi] range

print('\nNormalise Encoding:')
print_ts_relang_norm_val(1.5*np.pi)
print_ts_relang_norm_val(-0.5*np.pi)



Encoding:
0 -> 0.0 (π*0.0)
0.5 -> 0.785 (π/4.002)
0.7 -> 1.1 (π/2.856)
0.25 -> 0.393 (π/7.994)
1.2 -> 1.885 (π/1.667)
-0.7 -> -1.1 (π/-2.856)

Decoding:
3.142 (π/1.0) -> 2.0
0.0 (π*0.0) -> 0.0
0.785 (π/4.0) -> 0.5
-1.571 (π/-2.0) -> -1.0
0.393 (π/8.0) -> 0.25
-2.827 (π/-1.111) -> -1.8

Normalise Encoding:
4.712 -> 4.712
-1.571 -> -1.571


In [5]:
### Create some windows for training and validation
y_train_ts = np.array([
    [-0.3, -0.2, -0.1, 0],
    [0.1, 0.2, 0.3, 0.4],
    [0.5, 0.6, 0.7, 0.8]])

y_valid_ts = np.array([
    [-0.6, -0.5, -0.4, -0.3],
    [0.9, 0.8, 0.7, 0.6]])


### Encoding of TS windows which are 
y_train_enc, org_train_start = ts_relang_encode(y_train_ts)
y_valid_enc, org_valid_start = ts_relang_encode(y_valid_ts)

### Testing validation windows

y_train_dec = ts_relang_decode(org_train_start, ts_relang_norm(y_train_enc))
print('\nTraining windows before encoding:\n', y_train_ts)
print('\nTraining windows after encoding:\n', y_train_enc)
print('\nTraining windows org start:\n', org_train_start)
print('\nTraining windows after decoding:\n', y_train_dec)

y_valid_dec = ts_relang_decode(org_valid_start, y_valid_enc)
print('\nValidation windows before encoding:\n', y_valid_ts)
print('\nValidation windows after encoding:\n', y_valid_enc)
print('\nValidation windows org start:\n', org_valid_start)
print('\nValidation windows after decoding:\n', y_valid_dec)


Training windows before encoding:
 [[-0.3 -0.2 -0.1  0. ]
 [ 0.1  0.2  0.3  0.4]
 [ 0.5  0.6  0.7  0.8]]

Training windows after encoding:
 [[-0.9424778  -0.62831853 -0.31415927  0.        ]
 [ 0.31415927  0.62831853  0.9424778   1.25663706]
 [ 1.57079633  1.88495559  2.19911486  2.51327412]]

Training windows org start:
 [-0.3  0.1  0.5]

Training windows after decoding:
 [[-0.3 -0.2 -0.1  0. ]
 [ 0.1  0.2  0.3  0.4]
 [ 0.5  0.6  0.7  0.8]]

Validation windows before encoding:
 [[-0.6 -0.5 -0.4 -0.3]
 [ 0.9  0.8  0.7  0.6]]

Validation windows after encoding:
 [[-1.88495559 -1.57079633 -1.25663706 -0.9424778 ]
 [ 2.82743339  2.51327412  2.19911486  1.88495559]]

Validation windows org start:
 [-0.6  0.9]

Validation windows after decoding:
 [[-0.6 -0.5 -0.4 -0.3]
 [ 0.9  0.8  0.7  0.6]]


In [6]:
!pip list | grep -e qiskit

qiskit                        0.45.2
qiskit-aer-gpu                0.13.2
qiskit-algorithms             0.2.2
qiskit-dynamics               0.4.3
qiskit-finance                0.4.0
qiskit-ibm-provider           0.6.3
qiskit-ibm-runtime            0.11.3
qiskit-ibmq-provider          0.20.2
qiskit-machine-learning       0.7.1
qiskit-nature                 0.7.1
qiskit-optimization           0.6.0
qiskit-qasm3-import           0.4.1
qiskit-sphinx-theme           1.12.1
qiskit-terra                  0.45.2
